In [ ]:
from torchvision.models.segmentation import deeplabv3
import torch.nn as nn
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from utils import data
import pandas as pd
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
import config
from tqdm import tqdm
import kornia
import matplotlib.pyplot as plt
import numpy as np


from models.msnet import MSNet
from torch.utils.data import DataLoader
from utils.data import create_dataloaders
from utils.train import train_epoch



In [ ]:
#Should be cuda:0 in colab and cpu in local.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
DATA_PATH = config.PATH_TO_DATA


In [ ]:
loader_params = {
    "batch_size" : 4,
    "batch_transforms" : True,
    "num_workers" : 1
}
train_loader, test_loader, val_loader = create_dataloaders(DATA_PATH, **loader_params)


In [ ]:
%load_ext autoreload
%autoreload 2
max_epochs = 1
params = {'batch_size': 4,
          'shuffle': True}
model = MSNet(num_classes=2)
learning_rate = 0.001 
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
loss_history, accuracy_history = train_epoch(model=model,device=device,train_loader=train_loader,optimizer=optimizer,epoch=1,criterion=criterion)


In [ ]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

#tqdm.close()

# Assuming you have a trained model, test data, and labels
model.eval()  # Set the model to evaluation mode
y_true = []  # True labels
y_pred = []  # Predicted labels

with torch.no_grad():
    for local_batch, local_labels in tqdm(test_loader, total=len(test_loader.torch_loader)):
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        outputs = model(local_batch)
        _, predicted = torch.max(outputs, 1)
        
        y_true.extend(local_labels.cpu().numpy())  # Collect true labels
        y_pred.extend(predicted.cpu().numpy())  # Collect predicted labels

# Calculate precision, recall, and F1 score
y_pred = np.array(y_pred).flatten()
y_true = np.array(y_true).flatten()
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')